In [1]:
import numpy as np
import pinocchio as pin

In [2]:
%%capture
!pip3 install robot_descriptions
from robot_descriptions import (
    z1_mj_description,
    z1_description,
    skydio_x2_mj_description,
    skydio_x2_description,
    panda_description,
    panda_mj_description
)

In [3]:
ROBOT_URDF = skydio_x2_description.URDF_PATH
ROBOT_XML = skydio_x2_mj_description.MJCF_PATH

ROBOT_URDF = z1_description.URDF_PATH
ROBOT_XML = z1_mj_description.MJCF_PATH

ROBOT_URDF = panda_description.URDF_PATH
ROBOT_XML = panda_mj_description.MJCF_PATH

In [4]:
pin_model = pin.buildModelFromUrdf(ROBOT_URDF)
pin_data = pin_model.createData()

In [5]:
q = np.random.randn(pin_model.nq)
dq = np.random.randn(pin_model.nv)
ddq = np.random.randn(pin_model.nv)
# q[:3] = np.zeros(3)
# dq[:3] = np.zeros(3)
# ddq[:3] = np.zeros(3)

if pin_model.nq != pin_model.nv:
    q[3:7] /= np.linalg.norm(q[3:7])
    
pin_data.q = q.copy()
pin_data.dq = dq.copy()
pin_data.ddq = ddq.copy()

### Regressor in Mujoco

In [6]:
import mujoco

In [7]:
0# Load model, make data
mj_model = mujoco.MjModel.from_xml_path(ROBOT_XML)
mj_data = mujoco.MjData(mj_model)
mj_model.opt.enableflags |= mujoco.mjtEnableBit.mjENBL_ENERGY
body_offset = np.where(mj_model.body_dofadr == 0)[0][0]
# model.energy.setflags(enable=True)

# Set the state to the "squat" keyframe, call mj_forward.
# key = model.key('hover').id
# mujoco.mj_resetDataKeyframe(model, data, key)

q_muj = q.copy()
dq_muj = dq.copy()
ddq_muj = ddq.copy()

if mj_model.nq != mj_model.nv:
    q_muj[[3, 4, 5, 6]] = q[[6, 3, 4, 5]]
    mj_data.qpos = q_muj
    mujoco.mj_kinematics(mj_model, mj_data)
    rotation = mj_data.xmat[1].reshape(3, 3).copy()
    dq_muj[:3] = rotation@dq_muj[:3]
    ddq_muj[:3] = rotation@ddq_muj[:3]# + np.cross(dq_muj[:3], dq_muj[3:]))

mj_data.qpos = q_muj
mj_data.qvel = dq_muj
mj_data.qacc = ddq_muj 

In [8]:
mujoco.mj_inverse(mj_model, mj_data)

In [9]:
def Y_body(v_lin, v_ang, a_lin, a_ang):
    # this is identical to the pinnochio: pin.bodyRegressor(*[v_lin, v_ang], *[a_lin, a_ang])
    v1, v2, v3 = v_lin
    v4, v5, v6 = v_ang

    a1, a2, a3 = a_lin
    a4, a5, a6 = a_ang

    # fmt: off
    Y = np.array([
        [a1 - v2*v6 + v3*v5, -v5**2 - v6**2, -a6 + v4*v5, a5 + v4*v6, 0, 0, 0, 0, 0, 0],
        [a2 + v1*v6 - v3*v4, a6 + v4*v5, -v4**2 - v6**2, -a4 + v5*v6, 0, 0, 0, 0, 0, 0],
        [a3 - v1*v5 + v2*v4, -a5 + v4*v6, a4 + v5*v6, -v4**2 - v5**2, 0, 0, 0, 0, 0, 0],
        [0, 0, a3 - v1*v5 + v2*v4, -a2 - v1*v6 + v3*v4, a4, a5 - v4*v6, -v5*v6, a6 + v4*v5, v5**2 - v6**2, v5*v6],
        [0, -a3 + v1*v5 - v2*v4, 0, a1 - v2*v6 + v3*v5, v4*v6, a4 + v5*v6, a5, -v4**2 + v6**2, a6 - v4*v5, -v4*v6],
        [0, a2 + v1*v6 - v3*v4, -a1 + v2*v6 - v3*v5, 0, -v4*v5, v4**2 - v5**2, v4*v5, a4 - v5*v6, a5 + v4*v6, a6]
    ])
    # fmt: on

    return Y


def mj_bodyRegressor(mj_model, mj_data, body_id):
    # TODO:
    # Clarify more about transformations:
    # not working properly with floating joint
    # https://mujoco.readthedocs.io/en/stable/programming/simulation.html#coordinate-frames-and-transformations
    # https://royfeatherstone.org/teaching/index.html
    # https://royfeatherstone.org/teaching/2008/slides.pdf

    velocity = np.zeros(6)
    accel = np.zeros(6)
    _cross = np.zeros(3)

    mujoco.mj_objectVelocity(mj_model, mj_data, 2, body_id, velocity, 1)
    mujoco.mj_rnePostConstraint(mj_model, mj_data)
    mujoco.mj_objectAcceleration(mj_model, mj_data, 2, body_id, accel, 1)
    # rotation = mj_data.xmat[body_id].reshape(3, 3).copy()

    v, w = velocity[3:], velocity[:3]
    dv, dw = accel[3:], accel[:3]  # dv - classical acceleration, already containt g
    mujoco.mju_cross(_cross, w, v)
    
    # in floating body, should be canceled?
    dv -= _cross
    # print(v, w)
    # print(dv, dw)
    
    Y = Y_body(v, w, dv, dw)
    return Y


def mj_jointRegressor(mj_model, mj_data, body_offset = 0):
    # TODO:
    # Investigate spatial tranfromation mju_transformSpatial t
    # not working properly with floating joint
    
    njoints = mj_model.njnt
    body_regressors = np.zeros((6 * njoints, njoints * 10))
    col_jac = np.zeros((6 * njoints, mj_model.nv))
    jac_lin = np.zeros((3, mj_model.nv))
    jac_rot = np.zeros((3, mj_model.nv))

    for i in range(njoints):
        # calculate cody regressors
        body_regressors[6 * i : 6 * (i + 1), 10 * i : 10 * (i + 1)] = mj_bodyRegressor(
            mj_model, mj_data, i + body_offset
        )  
        
        mujoco.mj_jacBody(mj_model, mj_data, jac_lin, jac_rot, i + body_offset)


        # Calculate jacobians
        rotation = mj_data.xmat[i + body_offset].reshape(3, 3).copy()
        col_jac[6 * i : 6 * i + 3, :] = rotation.T @ jac_lin.copy()
        col_jac[6 * i + 3 : 6 * i + 6, :] = rotation.T @ jac_rot.copy()

    joint_regressor = col_jac.T @ body_regressors
    return joint_regressor


### Compare Joint regressors

In [10]:
joint_regressor_muj = mj_jointRegressor(mj_model, mj_data, body_offset=body_offset)

In [11]:
pin.computeAllTerms(pin_model, pin_data, q, dq)
pin.forwardKinematics(pin_model, pin_data, q, dq, ddq)
joint_regressor_pin = pin.computeJointTorqueRegressor(pin_model, pin_data, q, dq, ddq)

In [12]:
joint_regressor_muj

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         8.16459288e-01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  2.06451814e-17,  7.86819982e-01,
        -3.08688926e-01,  2.96393058e-02, -1.42943381e+00,
         3.21430214e-01,  8.09266348e-17,  7.85686961e-02,
        -4.52824503e-01,  3.04727266e-01,  4.97270229e-01,
        -1.55846676e+00, -1.06493980e+00,  2.34528674e+00,
         1.43298893e+00, -9.64326790e-01,  3.62854517e-01,
         5.96429611e-02,  1.37962675e-02,  5.73740422e-01,
        -3.92584799e-01, -1.48254796e+00, -1.58063479e+00,
         2.86935713e-01,  1.31252265e+00,  1.83559424e+00,
         2.88447397e+00,  4.57054390e-01, -8.33545013e-01,
         1.00704026e+00,  1.52001723e+00, -4.25714085e-01,
        -4.53401785e+00,  1.82764010e+00, -1.39815610e-01,
        -2.46768926e+00,  8.14059214e-01,  4.57054390e-0

In [13]:
np.allclose(joint_regressor_muj, joint_regressor_pin)

False

In [33]:
np.allclose(joint_regressor_muj[:7, :70],joint_regressor_pin[:7,:70])

True

### Compare Body regressors

In [15]:
body_id = 3

In [16]:
body_regressor_muj = mj_bodyRegressor(mj_model, mj_data, body_id+1)

In [17]:
v_spatial = pin.getVelocity(pin_model, pin_data, body_id, pin.LOCAL)
a_spatial = np.array(pin.getAcceleration(pin_model, pin_data, body_id, pin.LOCAL))
rotation = pin_data.oMi[body_id].rotation
a_spatial[:3] -= rotation.T @ pin_model.gravity.linear
print(v_spatial, a_spatial)

  v =   0.252345   0.276646 8.3134e-17
  w = -0.875462  0.798561  0.828887
 [-6.1566225   6.15045947 -2.14996718  1.29484439  2.57490822 -1.58760856]


In [18]:
body_regressor_pin = pin.bodyRegressor(pin.Motion(v_spatial), pin.Motion(a_spatial))

In [19]:
mj_model.nbody

12

In [20]:
body_regressor_pin

array([[-6.38593055, -1.32475317,  0.88849891,  1.84924985,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 6.35962517, -2.28671821, -1.45348597, -0.63292762,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-2.59367319, -3.3005666 ,  1.95676116, -1.40413295,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -2.59367319, -6.35962517,  1.29484439,
         3.3005666 , -0.66191677, -2.28671821, -0.04935302,  0.66191677],
       [ 0.        ,  2.59367319,  0.        , -6.38593055, -0.72565838,
         1.95676116,  2.57490822, -0.07937978, -0.88849891,  0.72565838],
       [ 0.        ,  6.35962517,  6.38593055,  0.        ,  0.69910965,
         0.1287328 , -0.69910965,  0.63292762,  1.84924985, -1.58760856]])

In [21]:
np.allclose(body_regressor_muj, body_regressor_pin)

True